<a href="https://colab.research.google.com/github/nayan-datta-barua/Deep_learning_projects_2023/blob/main/Patato_disease_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import models,layers
%matplotlib inline

In [ ]:
# !unzip "/content/archive.zip";

In [ ]:
root='/content/Potato';

In [ ]:
image_size=256
batch_size=32
chanels=3

In [ ]:
df = tf.keras.preprocessing.image_dataset_from_directory(root,shuffle=True,image_size=(image_size,image_size),batch_size=batch_size)

In [ ]:
df.class_names

In [ ]:
plt.figure(figsize=(10,10))
for image_batch,label_batch in df.take(1):   # it counts batch_size() thats means 1 batch = 32 
     print(image_batch.shape)
     print(label_batch.numpy())
     for i in range(12):
       ax =plt.subplot(3,4,i+1)
       plt.imshow(image_batch[i].numpy().astype("uint8"))
       plt.title(df.class_names[label_batch[i]])
       plt.axis("off")
      
        

In [ ]:
# separate dataset traning and test and validation dataset
def get_seperate_dataset(ds,train_split=.8,val_split=0.1,test_split=0.1,shuffle=True,shuffle_size=10000):
  assert(train_split + val_split + test_split) ==1
  ds_size=len(ds)
  train_size=int(ds_size*train_split)
  val_size=int(ds_size*test_split)

  train_ds= ds.take(train_size)
  val_ds=ds.skip(train_size).take(val_size)
  test_ds = ds.skip(train_size).skip(val_size)


  
  return train_ds,val_ds,test_ds


In [ ]:
train_ds,val_ds,test_ds =get_seperate_dataset(df)

In [ ]:
print(len(train_ds))
print(len(val_ds))
print(len(test_ds))


In [ ]:
train_ds=train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds=test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds=val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)


In [ ]:
resize_and_rescale = tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(256,256),
    layers.experimental.preprocessing.Rescaling(.1/256)
])

In [ ]:
data_augmentation=tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip('horizontal_and_vertical'),
    layers.experimental.preprocessing.RandomRotation(0.2),
])

In [ ]:
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D
input_shape=(batch_size,image_size,image_size,chanels)
n_classes=3
model=models.Sequential([
    resize_and_rescale,
    data_augmentation,
    layers.Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=input_shape),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(64,activation='relu'),
    layers.Dense(n_classes,activation='softmax'),
    


    
])
model.build(input_shape=input_shape)

In [ ]:
model.summary()

In [ ]:
from keras.api._v2.keras import metrics
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy'],
)

In [ ]:
history=model.fit(
    train_ds,
    batch_size=batch_size,
    validation_data=val_ds,
    verbose=1,
    epochs=50
)

In [ ]:
history2=model.evaluate(test_ds)

In [ ]:
history2

In [ ]:
history.history.keys()

In [ ]:
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']

In [ ]:
plt.figure(figsize=(8,8))
plt.subplot(1,2,1)
plt.plot(range(50),acc,label="Tranning Accuracy")
plt.plot(range(50),val_acc,label="validation Accuracy")
plt.legend(loc='lower right')
plt.title("traning and validation accuracy")

plt.figure(figsize=(8,8))
plt.subplot(1,2,1)
plt.plot(range(50),loss,label="Tranning Accuracy")
plt.plot(range(50),val_loss,label="validation Accuracy")
plt.legend(loc='lower right')
plt.title("traning and validation accuracy")


In [ ]:
pd.DataFrame(history.history).plot(figsize=(8,5))
plt.grid(True)
plt.gca().set_ylim(0,1)

In [ ]:
class_name=df.class_names

In [ ]:
for image_batch,label_batch in df.take(2):   # it counts batch_size() thats means 1 batch = 32 
     first_image=image_batch[0].numpy().astype('uint8')
     plt.imshow(first_image)
     print("actual label:", class_name[label_batch[0].numpy()])
     predict=model.predict(image_batch)
     print("predict label :",class_name[np.argmax(predict[0])])

In [ ]:
def predict(model,images):
  img_array=tf.keras.preprocessing.image.img_to_array(images[i].numpy())
  img_array=tf.expand_dims(img_array,0)  # create a batch

  prediction=model.predict(img_array)
  prediction_class =class_name[np.argmax(prediction[0])]
  confidance= round(100 * (np.max(prediction[0])),2)
  return prediction_class , confidance

In [ ]:
import numpy 

In [ ]:
# from keras.utils.dataset_utils import labels_to_dataset
plt.figure(figsize=(12,15))
for images,labels in test_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3,3,i+1)
    plt.imshow(images[i].numpy().astype('uint8'))
    prediction_class , confidance =predict(model,images)

    actual_class =class_name[labels[i]]
    plt.title(f"actual class: {actual_class},\n Predicted class {prediction_class} ,\n confidence={confidance}")
    plt.axis("off")